In [3]:
!pip install GoogleNews

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install pandas
!pip install openpyxl
!pip install requests
!pip install beautifulsoup4
!pip install newspaper3k
!pip install selenium
!pip install openai
!pip install serpapi

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=a139028e1ff25cbab59e3b46f717cb62447ca845de77a0ca4e1daf9cd67ea93b
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=291702234c4fac824b3104b8a9f2ee532d6e9d2837395e97b49578b8ee92c44c
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [5]:
!pip install googlesearch-python
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=4b9c62eb35b6f893b1ba8ee918bba2f015f61fb1b8296585c58478db50d20020
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [6]:
!pip install "httpx[http2]"

In [7]:
!pip install "lxml[html_clean]"

In [8]:
from tabulate import tabulate
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import random
from datetime import datetime
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
import pandas as pd
from openai import OpenAI
import json

# ================================================================
# Keys
# ================================================================
OPENAI_API_KEY = ""
SerpAPI_tkn1 = ""

client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
!pip install GoogleNews

In [ ]:

# ================================================================
# 🌐 PARTE 1: FUNCIONES DE SCRAPING (Tu código original)
# ================================================================

def buscar_noticias_generales(nombre_fuente, url_base, termino_busqueda, max_resultados=15):
    """
    🔍 EXPLICACIÓN:
    Esta función busca noticias en sitios web específicos (El Tiempo, ADE, FECODE).

    - Construye una URL de búsqueda usando el término
    - Hace una petición HTTP para obtener el HTML
    - Usa BeautifulSoup para extraer información de los artículos
    - Devuelve una lista de diccionarios con los datos de cada noticia
    """
    print(f"\n🔍 Buscando en {nombre_fuente}: {termino_busqueda}")
    datos = []

    # quote_plus convierte espacios en + para URLs (ej: "paro maestros" → "paro+maestros")
    termino_codificado = quote_plus(termino_busqueda)
    url_busqueda = f"{url_base}{termino_codificado}"

    # User-Agent simula que somos un navegador real (algunos sitios bloquean bots)
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()  # Lanza error si status code != 200

        # BeautifulSoup parsea el HTML para poder buscar elementos
        soup = BeautifulSoup(response.content, 'html.parser')

        # find_all busca todos los tags <article> (donde suelen estar las noticias)
        articulos = soup.find_all('article', limit=max_resultados)
        print(f"📰 Encontrados {len(articulos)} artículos en {nombre_fuente}")

        for articulo in articulos:
            # Extrae título (busca <h2>), descripción (<p>), link (<a>), fecha (<time>)
            titulo = articulo.find('h2').get_text(strip=True) if articulo.find('h2') else ""
            descripcion = articulo.find('p').get_text(strip=True) if articulo.find('p') else ""
            link_elem = articulo.find('a', href=True)
            link = link_elem['href'] if link_elem else ""

            # Si el link es relativo (ej: /noticia/123), lo convierte a absoluto
            if link and not link.startswith('http'):
                link = f"{url_base.rstrip('/')}/{link.lstrip('/')}"

            fecha_elem = articulo.find('time')
            fecha = fecha_elem.get_text(strip=True) if fecha_elem else 'N/A'

            # Solo guarda si tiene título válido
            if titulo and len(titulo) > 10:
                datos.append({
                    'titulo': titulo,
                    'fuente': nombre_fuente,
                    'fecha_publicacion': fecha,
                    'descripcion': descripcion,
                    'url': link,
                    'periodo_busqueda': termino_busqueda,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
    except Exception as e:
        print(f"❌ Error en {nombre_fuente}: {e}")

    return datos


def buscar_con_googlenews():
    """
    🔍 EXPLICACIÓN:
    Usa la librería GoogleNews para buscar noticias en español.

    - set_time_range limita resultados al año 2023
    - Hace múltiples búsquedas con diferentes queries
    - Devuelve lista de noticias encontradas
    """
    googlenews = GoogleNews(lang='es', encode='utf-8')
    googlenews.set_time_range('01/01/2023', '12/31/2023')

    queries = [
    # Originales
    '"paro de docentes" Bogotá "clases suspendidas 2023"',
    '(paro OR "cese de actividades 2023") AND (docentes OR maestros) AND Bogotá',
    '"estudiantes sin clase 2023" AND paro Bogotá 2023',
    '"suspensión de clases" AND docentes Bogotá',

    # Mejoradas
    '("paro de docentes" OR "cese de actividades") AND Bogotá AND ("clases suspendidas" OR "clases interrumpidas") AND 2023',
    '(paro OR "cese de actividades") AND (docentes OR maestros OR profesores) AND Bogotá AND 2023',
    '("estudiantes sin clase" OR "no hay clases" OR "interrupción académica") AND paro AND Bogotá AND 2023',
    '("suspensión de clases" OR "clases suspendidas" OR "no hay clases") AND (docentes OR maestros) AND Bogotá AND 2023'
]


    resultados = []
    for q in queries:
        googlenews.search(q)
        time.sleep(random.uniform(2, 4))  # Espera aleatoria para no ser bloqueado
        res = googlenews.result()

        for r in res:
            resultados.append({
                'titulo': r.get('title'),
                'fuente': r.get('media'),
                'fecha_publicacion': r.get('date'),
                'descripcion': '',
                'url': r.get('link'),
                'periodo_busqueda': q,
                'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

    print(f"✅ GoogleNews: {len(resultados)} resultados")
    return resultados


def buscar_con_serpapi(api_key):
    """
    🔍 EXPLICACIÓN:
    Usa SerpAPI (servicio de pago) para buscar en Google de forma programática.

    - Permite búsquedas avanzadas con operadores de Google
    - Devuelve resultados estructurados (organic_results)
    - Más confiable que el scraping directo
    """
    print("\n🔍 Buscando con SerpAPI...")
    queries = [
        '("suspensión de clases" OR "cese de actividades" OR "interrupción académica") Bogotá 2023',
        '("paro de maestros" OR "paro docente" OR "huelga de profesores") Bogotá 2023',
        '("interrupción de clases" OR "cancelación de clases") Bogotá 2023',
        '("sindicato de maestros" OR "FECODE" OR "ADE Bogotá") AND (paro OR huelga) 2023',
    ]

    resultados = []
    for query in queries:
        print(f" • {query}")
        params = {
            "engine": "google",
            "q": f"{query} after:2023-01-01 before:2023-12-31",
            "api_key": api_key,
            "num": 50,  # Número máximo de resultados por query
        }

        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            items = results.get("organic_results", [])

            for item in items:
                resultados.append({
                    'titulo': item.get('title'),
                    'fuente': 'Google (SerpAPI)',
                    'fecha_publicacion': '',
                    'descripcion': item.get('snippet', ''),
                    'url': item.get('link'),
                    'periodo_busqueda': query,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except Exception as e:
            print(f"❌ Error con SerpAPI: {e}")

    print(f"✅ SerpAPI: {len(resultados)} resultados")
    return resultados


# ================================================================
# 🤖 PARTE 2: ANÁLISIS CON IA (Tu código de OpenAI adaptado)
# ================================================================

def obtener_texto_completo(url):
    """
    📄 EXPLICACIÓN:
    Descarga el contenido completo de una URL y extrae solo el texto legible.

    - Elimina scripts, estilos, headers, footers (no son contenido útil)
    - stripped_strings elimina espacios innecesarios
    - Limita a 8000 caracteres para no exceder límites de OpenAI
    """
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        # Elimina elementos que no son contenido principal
        for s in soup(["script", "style", "noscript", "header", "footer", "aside", "form"]):
            s.extract()

        texto = " ".join(soup.stripped_strings)
        return texto[:8000]  # Máximo 8000 caracteres
    except Exception as e:
        return f"Error al obtener contenido: {e}"


def analizar_con_ia(titulo, descripcion, url):
    """
    🤖 EXPLICACIÓN (LA MÁS IMPORTANTE):
    Esta función envía la noticia a GPT-4 para que la analice siguiendo criterios estrictos.

    PROCESO:
    1. Intenta descargar el contenido completo de la URL
    2. Si falla, usa solo título + descripción, indicame si pudiste o no acceder al URL
    3. Construye un prompt detallado con instrucciones para la IA
    4. Envía el prompt a OpenAI (modelo gpt-4o-mini)
    5. Recibe respuesta en formato JSON
    6. Parsea el JSON y devuelve un diccionario con las características

    CARACTERÍSTICAS QUE ANALIZA:
    - es_paro_docente: ¿Hubo suspensión de clases?
    - organizaciones_sindicales: ¿Qué sindicatos u organizaciones políticas participaron?
    - duracion_dias: ¿Cuántos días duró?
    - ubicacion_bogota: ¿Ocurrió en Bogotá?
    - ubicacion_Colombia: ¿Abarcó todo el territorio Nacional=
    - costo_mencionado: ¿Se habla de costos económicos?
    - fecha_paro: ¿cuál es la fecha exacta del paro (día, mes, año)? 
    """
    print(f"  🤖 Analizando: {titulo[:60]}...")

    # Intenta obtener contenido completo de la URL
    contenido_completo = obtener_texto_completo(url) if url else ""

    # Si no pudo descargar el contenido, usa título + descripción
    if "Error" in contenido_completo or not contenido_completo:
        texto_analizar = f"Título: {titulo}\nDescripción: {descripcion}"
    else:
        texto_analizar = contenido_completo

    # ============================================
    # 📝 PROMPT PARA LA IA
    # ============================================
    # Este es el "cerebro" del análisis. Le dice a GPT exactamente qué debe hacer.
    prompt = f"""
Analiza la siguiente noticia en español sobre educación, sindicatos o protestas de docentes.

CRITERIOS ESTRICTOS:
- Un **paro docente verdadero** solo existe si hubo suspensión de clases programadas.
- Si fue marcha, manifestación o concentración SIN suspensión de clases, NO es paro.
- Debes explicar tu razonamiento.

Devuelve un JSON con estas claves:

- "es_paro_docente": true/false → true solo si hubo suspensión de clases
- "justificacion_paro": texto breve (2-4 líneas) explicando por qué es o no un paro
- "organizaciones_sindicales": lista de sindicatos mencionados u organizaciones políticas o partidos políticos (ej: ["FECODE", "ADE"]), o []
- "hay_suspension_clases": true/false → si se suspendieron clases
- "duracion_dias": número de días estimados, o null
- "razones_paro": resumen de las demandas principales
- "ubicacion_bogota": true/false → si ocurre en Bogotá/Cundinamarca
- "ubicacion_Colombia": true/false → si el paro abarca el territorio Nacional
- "costo_mencionado": monto económico si se menciona, o null
- "resumen": breve resumen (máx. 3 líneas)
- "fecha_paro": ¿cuál es la fecha exacta del paro (día, mes, año)? 
- "tipo_movilizacion": ¿cuál es el tipo de movilización, paro docente con las caracteristicas que ya vimos, paro de otro gremio, demostración, plantón, toma, marcha pacífica, etc?
- "ubicacion": además del país y ciudad, podemos identificar localidades? barrios? calles? edificios? colegios específicos? edificios del gobierno?
Devuelve SOLO el JSON válido, sin texto extra.

Texto a analizar:
{texto_analizar}
"""

    try:
        # ============================================
        # 🚀 LLAMADA A LA API DE OPENAI
        # ============================================
        # - model: versión de GPT a usar (gpt-4o-mini es rápida y barata)
        # - temperature: 0.1 = respuestas consistentes (menos creatividad)
        # - messages: formato de conversación (role: user/assistant)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0.1,
            messages=[{"role": "user", "content": prompt}],
        )

        # Extrae el texto de la respuesta
        respuesta_texto = response.choices[0].message.content.strip()

        # ============================================
        # 📊 PARSEO DEL JSON
        # ============================================
        # La IA devuelve un JSON como string, lo convertimos a diccionario
        try:
            # Intenta parsear directamente
            analisis = json.loads(respuesta_texto)
        except json.JSONDecodeError:
            # Si falla, busca el JSON dentro de bloques de código ```json```
            if "```json" in respuesta_texto:
                inicio = respuesta_texto.find("```json") + 7
                fin = respuesta_texto.find("```", inicio)
                json_str = respuesta_texto[inicio:fin].strip()
                analisis = json.loads(json_str)
            else:
                # Si todo falla, devuelve valores por defecto
                print("    ⚠️ No se pudo parsear el JSON")
                analisis = {
                    'es_paro_docente': False,
                    'justificacion_paro': 'Error al analizar',
                    'organizaciones_sindicales': [],
                    'hay_suspension_clases': False,
                    'duracion_dias': None,
                    'razones_paro': '',
                    'ubicacion_bogota': False,
                    'costo_mencionado': None,
                    'resumen': ''
                }

        return analisis

    except Exception as e:
        print(f"    ❌ Error en análisis IA: {e}")
        return {
            'es_paro_docente': False,
            'justificacion_paro': f'Error: {str(e)}',
            'organizaciones_sindicales': [],
            'hay_suspension_clases': False,
            'duracion_dias': None,
            'razones_paro': '',
            'ubicacion_bogota': False,
            'costo_mencionado': None,
            'resumen': ''
        }


# ================================================================
# 🚀 PARTE 3: EJECUCIÓN PRINCIPAL (Combina todo)
# ================================================================

if __name__ == "__main__":
    print("="*60)
    print("🔥 INICIANDO SCRAPER + ANÁLISIS IA")
    print("="*60)

    resultados_totales = []

    # ============================================
    # FASE 1: RECOPILACIÓN DE NOTICIAS
    # ============================================
    print("\n📰 FASE 1: Recopilando noticias...")

    # --- El Tiempo ---
    terminos_tiempo = ["paro maestros 2023", "paro nacional docentes 2023", "ADE 2023", "FECODE 2023"]
    for termino in terminos_tiempo:
        resultados_totales.extend(buscar_noticias_generales("El Tiempo", "https://www.eltiempo.com/buscar/", termino))
        time.sleep(random.uniform(3, 6))

    # --- ADE ---
    terminos_ade = ["paro docentes", "paro maestros", "Fecode", "educación Bogotá"]
    for termino in terminos_ade:
        resultados_totales.extend(buscar_noticias_generales("ADE", "https://adebogota.org/?s=", termino))
        time.sleep(random.uniform(3, 6))

    # --- FECODE ---
    terminos_fecode = ["paro nacional", "paro docentes", "maestros", "educación Colombia"]
    for termino in terminos_fecode:
        resultados_totales.extend(buscar_noticias_generales("FECODE", "https://www.fecode.edu.co/index.php/component/search/?searchword=", termino))
        time.sleep(random.uniform(3, 6))

    # --- GoogleNews ---
    resultados_totales.extend(buscar_con_googlenews())

    # --- SerpAPI ---
    resultados_totales.extend(buscar_con_serpapi(SerpAPI_tkn1))

    print(f"\n✅ Total de noticias recopiladas: {len(resultados_totales)}")

    # ============================================
    # FASE 2: ANÁLISIS CON IA
    # ============================================
    print("\n🤖 FASE 2: Analizando con IA...")

    for i, noticia in enumerate(resultados_totales, 1):
        print(f"\n[{i}/{len(resultados_totales)}]", end=" ")

        # Llama a la función de análisis IA
        analisis = analizar_con_ia(
            noticia['titulo'],
            noticia.get('descripcion', ''),
            noticia.get('url', '')
        )

        # Agrega los resultados del análisis al diccionario de la noticia
        noticia.update(analisis)

        # Espera 2 segundos entre llamadas (para no saturar la API de OpenAI)
        time.sleep(2)

    # ============================================
    # FASE 3: GUARDAR RESULTADOS EN CSV
    # ============================================
    print("\n💾 FASE 3: Guardando resultados...")

    df = pd.DataFrame(resultados_totales)

    # Reordena columnas para mejor visualización
    columnas_importantes = [
        'titulo', 'fuente', 'fecha_publicacion', 'url',
        'es_paro_docente', 'hay_suspension_clases', 'ubicacion_bogota',
        'duracion_dias', 'organizaciones_sindicales', 'razones_paro',
        'costo_mencionado', 'justificacion_paro', 'resumen',
        'descripcion', 'periodo_busqueda', 'fecha_extraccion'
    ]

    # Solo incluye columnas que existan en el DataFrame
    columnas_finales = [col for col in columnas_importantes if col in df.columns]
    df = df[columnas_finales]

    # Guarda en CSV
    nombre_archivo = "noticias_paros_docentes_2023_ANALIZADO.csv"
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")

    print(f"✅ Archivo guardado: {nombre_archivo}")
    print(f"📊 Total de noticias: {len(df)}")

    # ============================================
    # ESTADÍSTICAS FINALES
    # ============================================
    print("\n" + "="*60)
    print("📈 ESTADÍSTICAS DEL ANÁLISIS")
    print("="*60)

    paros_verdaderos = df[df['es_paro_docente'] == True]
    print(f"🔴 Paros verdaderos (con suspensión de clases): {len(paros_verdaderos)}")

    en_bogota = df[df['ubicacion_bogota'] == True]
    print(f"📍 Eventos en Bogotá: {len(en_bogota)}")

    con_duracion = df[df['duracion_dias'].notna()]
    if len(con_duracion) > 0:
        print(f"⏱️ Duración promedio: {con_duracion['duracion_dias'].mean():.1f} días")

    print("\n🎉 ¡Análisis completado!")

🔥 INICIANDO SCRAPER + ANÁLISIS IA

📰 FASE 1: Recopilando noticias...

🔍 Buscando en El Tiempo: paro maestros 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: paro nacional docentes 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: ADE 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: FECODE 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en ADE: paro docentes
📰 Encontrados 0 artículos en ADE

🔍 Buscando en ADE: paro maestros
📰 Encontrados 0 artículos en ADE

🔍 Buscando en ADE: Fecode
📰 Encontrados 0 artículos en ADE

🔍 Buscando en ADE: educación Bogotá
📰 Encontrados 0 artículos en ADE

🔍 Buscando en FECODE: paro nacional
❌ Error en FECODE: 404 Client Error: Not Found for url: https://fecode.edu.co/component/search/?searchword=paro+nacional

🔍 Buscando en FECODE: paro docentes
❌ Error en FECODE: 404 Client Error: Not Found for url: https://fecode.edu.co/component/search/?searchword=paro+docentes

🔍 Buscando en FECO